<h1><center>Laboratorio 7: Aprendizaje Supervisado 🔮</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Matías Rojas y Mauricio Araneda
- Auxiliar: Ignacio Meza D.
- Ayudante: Rodrigo Guerra

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Vanessa Gaete
- Nombre de alumno 2:


### **Link de repositorio de GitHub:** `http://....`

## Temas a tratar

- Aprendizaje Supervisado
- Flujos de datos a través de `Pipelines`.

## Reglas

- Fecha de entrega: TBD
- **Grupos de 2 personas**
- **Ausentes** deberán realizar la actividad solos. 
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias. 
- Pueden usar cualquer matrial del curso que estimen conveniente.

## Objetivos principales del laboratorio

- Comprender el funcionamiento de clasificadores/regresores.
- Generar múltiples modelos predictivos.
- Comprender las ventajas de crear modelos en pipeline vs hacer las operaciones a mano.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre `DataFrames`.

### Importamos librerias utiles 😸

In [ ]:
# Libreria Core del lab.
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split 

# Pre-procesamiento
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.tree import DecisionTreeClassifier

# Metricas de evaluación
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score

#Libreria para plotear
!pip install --upgrade plotly
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 15.3 MB 6.6 MB/s 
  Attempting uninstall: plotly
    Found existing installation: plotly 5.5.0
    Uninstalling plotly-5.5.0:
      Successfully uninstalled plotly-5.5.0


https://www.kaggle.com/antoinekrajnc/soccer-players-statistics

# 1. Predicciones Futboleras

<p align="center">
  <img src="https://drive.google.com/uc?export=view&id=1BnO4tyh3vM2P199Ec9s3JjngQ4qQ9seP"
" width="300">
</p>


Tras el trágico despido de la mítica mascota de Maipú, Renacín decide adentrarse como consultor en el mercado futbolero, el cuál (para variar...) está cargado en especulaciones. 

Como su principal tarea será asesorar a los directivos de los clubes sobre cuál jugador comprar y cuál no, Renacín desea generar modelos predictivos que evaluén distintas características de los jugadores; todo con el fin de tomar decisiones concretas basadas en los datos.

Sin embargo, su condición de corporeo le impidió tomar la versión anterior de MDS7202, por lo que este motivo Renacín contrata a su equipo para lograr su objetivo final. Dado que aún tiene fuertes vínculos con la dirección de deportes de la municipalidad, el corporeo le entrega base de datos con las estadísticas de cada jugador para que su equipo empieze a trabajar ya con un dataset listo para ser usado.


**Los Datos**

Para este laboratorio deberán trabajar con los csv `statsplayers.csv` y `salarios.pickle`, donde deberán aplicar algoritmos de de aprendizaje supervisado (clasificación y regresión) en base a características que describen de jugadores de futbol.

Para comenzar cargue el dataset señalado y a continuación vea el reporte **`Player_Stats_Report.html`** (adjunto en la carpeta del enunciado) que describe las características principales del `DataFrame`.

In [ ]:
# Si usted está utilizando Colabolatory le puede ser útil este código para cargar los archivos.
try:
    from google.colab import drive
    drive.mount("/content/drive")
    path = 'Dirección donde tiene los archivos en el Drive'
except: 
    print('Ignorando conexión drive-colab')

Mounted at /content/drive


In [ ]:
df_players = pd.read_csv('stats_players.csv')
df_players

,Name,Nationality,National_Position,Club_Position,Height,Weight,Preffered_Foot,Age,Work_Rate,Weak_foot,...,Agility,Jumping,Heading,Shot_Power,Finishing,Long_Shots,Curve,Freekick_Accuracy,Penalties,Volleys
0,Cristiano Ronaldo,Portugal,LS,LW,185,80,Right,32,High / Low,4,...,90,95,85,92,93,90,81,76,85,88
1,Lionel Messi,Argentina,RW,RW,170,72,Left,29,Medium / Medium,4,...,90,68,71,85,95,88,89,90,74,85
2,Neymar,Brazil,LW,LW,174,68,Right,25,High / Medium,5,...,96,61,62,78,89,77,79,84,81,83
3,Luis Suárez,Uruguay,LS,ST,182,85,Right,30,High / Medium,4,...,86,69,77,87,94,86,86,84,85,88
4,Manuel Neuer,Germany,GK,GK,193,92,Right,31,Medium / Medium,4,...,52,78,25,25,13,16,14,11,47,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17583,Adam Dunbar,Republic of Ireland,NaN,Sub,183,82,Right,19,Medium / Medium,1,...,27,56,14,16,13,13,11,13,15,12
17584,Dylan McGoey,Republic of Ireland,NaN,Sub,185,80,Right,19,Medium / Medium,2,...,28,53,12,17,12,11,12,13,16,12
17585,Tommy Ouldridge,England,NaN,Res,173,61,Right,18,High / Medium,2,...,54,61,41,44,28,42,35,36,42,37
17586,Mark Foden,Scotland,NaN,Sub,180,80,Right,21,Medium / Medium,3,...,34,48,15,23,14,12,13,12,24,12


## 1.1 Predicción de Seleccionados Nacionales 

Como primera tarea, Renacín, intrigado por la posibilidad de saber qué tan reconocido es un jugador, le consulta a su equipo si es posible predecir si un jugador será o no seleccionado nacional a partir de sus estadísticas en el juego. 

<p align="center">
  <img src="https://thumbs.gfycat.com/ThornyFrankFoxhound-size_restricted.gif" width="500">
</p>

### 1.1.1 Generación de Labels para la Clasificación [Sin Puntaje]


Primero comience generando las labels para la clasificación. Para esto, trabaje sobre el atributo ```National_Position``` suponiendo que los valores nulos son jugadores no seleccionados para representar a su país. 

Hecho esto, ¿Cuantos ejemplos por cada clase se tienen? Comente lo que observa.



**Respuesta:**

In [ ]:
### Código aquí ###
#1 es para los que califican, 0 para los que no
df_players['label'] = df_players['National_Position'].apply(lambda x: 0 if pd.isna(x) else 1)
df_players

,Name,Nationality,National_Position,Club_Position,Height,Weight,Preffered_Foot,Age,Work_Rate,Weak_foot,...,Jumping,Heading,Shot_Power,Finishing,Long_Shots,Curve,Freekick_Accuracy,Penalties,Volleys,label
0,Cristiano Ronaldo,Portugal,LS,LW,185,80,Right,32,High / Low,4,...,95,85,92,93,90,81,76,85,88,1
1,Lionel Messi,Argentina,RW,RW,170,72,Left,29,Medium / Medium,4,...,68,71,85,95,88,89,90,74,85,1
2,Neymar,Brazil,LW,LW,174,68,Right,25,High / Medium,5,...,61,62,78,89,77,79,84,81,83,1
3,Luis Suárez,Uruguay,LS,ST,182,85,Right,30,High / Medium,4,...,69,77,87,94,86,86,84,85,88,1
4,Manuel Neuer,Germany,GK,GK,193,92,Right,31,Medium / Medium,4,...,78,25,25,13,16,14,11,47,11,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17583,Adam Dunbar,Republic of Ireland,NaN,Sub,183,82,Right,19,Medium / Medium,1,...,56,14,16,13,13,11,13,15,12,0
17584,Dylan McGoey,Republic of Ireland,NaN,Sub,185,80,Right,19,Medium / Medium,2,...,53,12,17,12,11,12,13,16,12,0
17585,Tommy Ouldridge,England,NaN,Res,173,61,Right,18,High / Medium,2,...,61,41,44,28,42,35,36,42,37,0
17586,Mark Foden,Scotland,NaN,Sub,180,80,Right,21,Medium / Medium,3,...,48,15,23,14,12,13,12,24,12,0


### 1.1.2 Camino a la clasificación [1 punto]

<p align="center">
  <img src="https://c.tenor.com/SoXZJHWnhKAAAAAC/pitbull-medel.gif" width="500">
</p>

Para preprocesar el dataset, genere un `ColumnTransformer` en donde especifique las transformaciones que hay que realizar para cada columna (por ejemplo StandarScaler, MinMaxScaler, OneHotEncoder, etc...) para que puedan ser utilizadas correctamente por el modelo predictivo y guardelo en algúna variable.

Luego, comente y justifique las transformaciones elegidas sobre cada una de las variables (para esto utilice el material ```Player_Stats_Report.html``` que viene en el zip del lab), al igual que las transformaciones aplicadas.

Hecho lo anterior, **defina al menos 3 pipelines para la clasificación**, en donde utilice el mismo ColumnTransformer definido anteriormente, pero que varie entre cada pipeline los clasificadores. 

Para seleccionar los clasificadores más adecuados, utilice la siguiente guía:

<p align="center">
  <img src="https://scikit-learn.org/stable/_static/ml_map.png" width="500">
</p>

<br/>

Con ella, comente y justifique cada una de las decisiones tomadas al momento de desarrollar su pipeline.

**Nota:** Si tiene problemas al utilizar OneHotEncoder puede utilizar el parámetro handle_unknown='ignore'. Esto hace que en la codificación se omitan las categorias que no aparecen en el entrenamiento. Pregunta dudosa (no tiene puntaje), ¿esto tiene sentido a nivel de modelos?.

**To-Do:**

- [ ] Genere un ```ColumnTransformer``` enfocado en preprocesar los datos.
- [ ] Indicar y Justificar que preprocesamiento utiliza sobre cada columna.
- [ ] Crear 3 ``pipelines`` con diferentes clasificadores.
- [ ] Para seleccionar los clasificadores base sus decisiones en la siguiente [guía](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html)
- [ ] No entrenar los ``pipelines`` aún.

**Nota:** No es necesario entrenar los clasificadores aún.

In [ ]:
from sklearn.preprocessing import  RobustScaler

ct = ColumnTransformer(
    [
        (
            "Scaler",
            StandardScaler(),
            [
                "Height",
                "Weight",
                "Age",
                "Ball_Control",
                "Dribbling",
                "Marking",
                "Sliding_Tackle",
                "Standing_Tackle",
                "Aggression",
                "Reactions",
                "Interceptions",
                "Vision",
                "Composure",
                "Crossing",
                "Short_Pass",
                "Long_Pass",
                "Acceleration",
                "Speed",
                "Stamina",
                "Strength",
                "Balance",
                "Agility",
                "Jumping",
                "Heading",
                "Shot_Power",
                "Finishing",
                "Long_Shots",
                "Curve",
                "Freekick_Accuracy",
                "Penalties",
                "Volleys"
            ],
        ),
        (
            "OneHot",
            OneHotEncoder(sparse=False, handle_unknown="ignore"),
            ["Name", "Nationality", "Club_Position", "Preffered_Foot", "Work_Rate"],
        ),
    ]
)



**Respuesta:**

In [ ]:
### Código aquí ###

Se usa Robust Scaler porque este nos permite transformar los valores a escala 0 y 1 funcionando correctamente con los outliers (aunque en verdad no hay pero para asegurarse).  



Como clasificadores se utilizará SVM lineal, SVM polinomial y KNN. No se usa Bayes porque este conlleva al supuesto de que no hay variables dependientes, sin embargo, como se puede ver en la matriz de corelación del archivo Player Stats, hay varias variables que están correlacionadas entre sí. Dados los clasificadores a utilizar, tambien se aplica OneHotEncoding a las variables que son caracteristicas.


### 1.1.3 Entrenemos los pipelines [1 punto]

<p align="center">
  <img src="https://i.gifer.com/7Cc8.gif" width="350">
</p>


Ahora, entrene los `pipeline` generados en los pasos anteriores. Para esto, primero separe los datos de entrenamiento en un conjunto de entrenamiento y de prueba  (la proporción queda a su juicio).

En este paso, seleccione los ejemplos de forma aleatoria e intente mantener la distribución original de labels de cada clase en los conjuntos de prueba/entrenamiento. (vea la documentación de `train_test_split`).


Luego, entrene los pipelines

Una vez entrenado su modelo, evalue su rendimiento a través de diferentes métricas, comentando que significa cada uno de los valores obtenidos. Puede usar la función `classification_report` para corroborar sus resultados.

- ¿Qué implican los valores de accuracy, precisión y recall de la clase positiva (la que indica que un jugador es seleccionado nacional)?
- ¿Podrían mejorarse los resultados?, ¿Cómo?
- ¿Influye la cantidad de ejemplos por clase?

**To-Do**:

- [ ] Separar el conjunto de datos en entrenamiento y de prueba.
- [ ] Entrenar los `pipelines`.
- [ ] Utilizar las métricas para evaluar los modelos generados.


**Respuesta:**

In [ ]:
### Código aquí ###


# Holdout
from sklearn.model_selection import train_test_split

features = df_players.drop(columns=["National_Position", "label"])
labels = df_players.loc[:, "label"]


display(features)
display(labels)



,Name,Nationality,Club_Position,Height,Weight,Preffered_Foot,Age,Work_Rate,Weak_foot,Skill_Moves,...,Agility,Jumping,Heading,Shot_Power,Finishing,Long_Shots,Curve,Freekick_Accuracy,Penalties,Volleys
0,Cristiano Ronaldo,Portugal,LW,185,80,Right,32,High / Low,4,5,...,90,95,85,92,93,90,81,76,85,88
1,Lionel Messi,Argentina,RW,170,72,Left,29,Medium / Medium,4,4,...,90,68,71,85,95,88,89,90,74,85
2,Neymar,Brazil,LW,174,68,Right,25,High / Medium,5,5,...,96,61,62,78,89,77,79,84,81,83
3,Luis Suárez,Uruguay,ST,182,85,Right,30,High / Medium,4,4,...,86,69,77,87,94,86,86,84,85,88
4,Manuel Neuer,Germany,GK,193,92,Right,31,Medium / Medium,4,1,...,52,78,25,25,13,16,14,11,47,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17583,Adam Dunbar,Republic of Ireland,Sub,183,82,Right,19,Medium / Medium,1,1,...,27,56,14,16,13,13,11,13,15,12
17584,Dylan McGoey,Republic of Ireland,Sub,185,80,Right,19,Medium / Medium,2,1,...,28,53,12,17,12,11,12,13,16,12
17585,Tommy Ouldridge,England,Res,173,61,Right,18,High / Medium,2,2,...,54,61,41,44,28,42,35,36,42,37
17586,Mark Foden,Scotland,Sub,180,80,Right,21,Medium / Medium,3,1,...,34,48,15,23,14,12,13,12,24,12


0        1
1        1
2        1
3        1
4        1
        ..
17583    0
17584    0
17585    0
17586    0
17587    0
Name: label, Length: 17588, dtype: int64

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    features, labels, test_size=0.33, shuffle=True, stratify=labels
)

In [ ]:
X_train_preprocessed = pd.DataFrame(
    ct.fit_transform(X_train),
    columns=np.concatenate(
        [ct.transformers_[0][2], ct.transformers_[1][1].get_feature_names_out()], axis=0
    ),
)

X_train_preprocessed



,Height,Weight,Age,Ball_Control,Dribbling,Marking,Sliding_Tackle,Standing_Tackle,Aggression,Reactions,...,Preffered_Foot_Right,Work_Rate_High / High,Work_Rate_High / Low,Work_Rate_High / Medium,Work_Rate_Low / High,Work_Rate_Low / Low,Work_Rate_Low / Medium,Work_Rate_Medium / High,Work_Rate_Medium / Low,Work_Rate_Medium / Medium
0,-1.212025,-0.475599,1.829042,0.183308,0.384449,-1.258863,-1.090995,-0.474790,2.009045,-0.193687,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,-0.165788,-0.039943,0.758655,0.954772,0.753613,1.290989,1.415850,1.355435,1.323171,1.419869,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,-0.913100,-1.056474,-0.097655,0.776742,0.542662,0.502853,0.905197,0.989390,0.923079,0.882017,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,1.328837,-0.039943,-0.311732,-0.113409,-1.197683,1.151906,0.858773,0.989390,-0.048575,-0.301257,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,1.627762,2.138337,0.758655,-2.368457,-2.410651,-1.722473,-1.648072,-1.755948,-1.134541,0.236595,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11778,-0.913100,-0.766037,1.186810,-0.172752,-0.142929,0.641936,0.673081,0.669101,-0.048575,0.129024,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
11779,-0.315250,-0.185162,-1.382119,0.301995,0.542662,-0.795254,-1.137418,-0.978102,0.408674,-1.807243,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
11780,0.432062,1.557462,1.186810,0.480025,0.120760,1.569155,1.369427,1.401191,1.380327,1.527439,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
11781,0.581525,0.976587,-0.525810,-0.232095,-0.353879,0.502853,0.487389,0.486078,0.637298,-0.731539,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix

knn_pipe = Pipeline(
    [
        ("preprocesamiento", ct), 
        ("knn", KNeighborsClassifier(n_neighbors=3))
    ]
)

knn_pipe.fit(X_train, y_train)
y_pred = knn_pipe.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_pred, y_test))





[[5443    7]
 [ 344   11]]
              precision    recall  f1-score   support

           0       1.00      0.94      0.97      5787
           1       0.03      0.61      0.06        18

    accuracy                           0.94      5805
   macro avg       0.51      0.78      0.51      5805
weighted avg       1.00      0.94      0.97      5805



Este clasificador no esta prediciendo bien los casos donde el fultbolista sí debiera ser considerado, acertando tan solo en un 11% de los casos. Se probó con otros valores para los n_neighbours y el valor empeoraba, asiq ue se dejo en 3.

In [ ]:


from sklearn.svm import SVC

svc_pipe = Pipeline([
    ("preprocesamiento", ct), 
    ("svc", SVC(kernel="linear"))
])

# Entrenamos
svc_pipe.fit(X_train, y_train)

# Predecimos
y_pred = svc_pipe.predict(X_test)

# Evaluamos
print("Matriz de confusión\n", confusion_matrix(y_pred, y_test))
print(classification_report(y_pred, y_test))



Matriz de confusión
 [[5394  246]
 [  56  109]]
              precision    recall  f1-score   support

           0       0.99      0.96      0.97      5640
           1       0.31      0.66      0.42       165

    accuracy                           0.95      5805
   macro avg       0.65      0.81      0.70      5805
weighted avg       0.97      0.95      0.96      5805



Con este modelo se puede ver que le va mucho mejor al clasificador con los jugadores que sí debiesen ser considerados. Sin embargo, aún su precisión es muy baja, obteniendo un mal clasificador de todas formas. 

In [ ]:
from sklearn.svm import SVC

svc_pipe_rbf = Pipeline([
    ("preprocesamiento", ct), 
    ("svc", SVC(kernel="rbf")
    )
])

# Entrenamos
svc_pipe_rbf.fit(X_train, y_train)

# Predecimos
y_pred = svc_pipe_rbf.predict(X_test)

# Evaluamos
print("Matriz de confusión\n", confusion_matrix(y_pred, y_test))
print(classification_report(y_pred, y_test))

Matriz de confusión
 [[5450  353]
 [   0    2]]
              precision    recall  f1-score   support

           0       1.00      0.94      0.97      5803
           1       0.01      1.00      0.01         2

    accuracy                           0.94      5805
   macro avg       0.50      0.97      0.49      5805
weighted avg       1.00      0.94      0.97      5805



Utilizando rbf se obtienen resultados incluso peores que los anteriores. Logrando clasificar bien  a un 0% de los valores que debian ser etiquetados como 1. 

## 1.2 Predicción de posiciones de jugadores [2 puntos]

<p align="center">
  <img src="https://pbs.twimg.com/media/E1rfA1aWEAYU6Ny.jpg" width="500">
</p>

En una nueva jornada de desmesuradas transacciones deportivas, Renacín escuchó a sus colegas discutir acerca de que el precio de cada jugador depende en gran medida de la posición en la cancha en la que juega. Y además, que hay bastantes jugadores nuevos que no tienen muy claro en que posición verdaderamente brillarían, por lo que actualmente puede que actualmente estén jugando en posiciones sub-optimas. 

Viendo que los resultados del primer análisis no son tan esperanzadores, el corporeo los comanda a cambiar su tarea: ahora, les solicita que construyan un clasificador enfocado en predecir la mejor posición de los jugadores en la cancha según sus características. 

Para lograr esto, primero, les pide que etiqueten de la siguiente manera los valores que aparecen en el atributo ```Club_Position```, pidiendo que agrupen los valores en los siguientes grupos:

**Nota**:  Renacín les recalca que **no deben utilizar los valores ```Sub``` y ```Res``` de esta columna**.

```python
ataque = ['ST', 'CF'] 
central_ataque = ['RW', 'CAM', 'LW'] 
central = ['RM', 'CM', 'LM'] 
central_defensa = ['RWB', 'CDM', 'LWB']
defensa = ['RB', 'CB', 'LB']
arquero = ['GK']
```

Cabe señalar que al igual como lo realizado con la clasificación binaria, deberá justificar en base a la [guía](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html) la elección del clasificador y se deben comentar los resultados obtenidos en la clasificación.

**Nota:** Clasifique solamente con las clases señaladas, si observa mas clases eliminelas de la clasficación.

**To-Do**:

- [ ] Aplique las etiquetas descritas anteriormente en cada uno de los valores señalados en esta sección.
- [ ] Cuente cuantos por clase quedan.
- [ ] Entrene el nuevo pipeline y ejecute una evaluación de este.  
- [ ] Comente los resultados obtenidos.

**Respuesta:**

In [ ]:
### Código aquí ###



def labeling(x):
  if x == "ST" or x == "CF":
    return 0
  elif x == "RW" or x == "CAM" or x == "LW":
    return 1
  elif x == "RM" or x == "CM" or x == "LM":
    return 2
  elif x == "RWB" or x == "CDM" or x == "LWB":
    return 3
  elif x == "RB" or x == "CB" or x == "LB":
    return 4
  elif x == "GK":
    return 5
  else:
    return None


df_position = df_players.copy()
df_position['label'] = df_players['Club_Position'].apply(lambda x: labeling(x))
df_position = df_position.dropna() #Eliminar columnas que no pertenecen a la clasificacion indicada

df_position

,Name,Nationality,National_Position,Club_Position,Height,Weight,Preffered_Foot,Age,Work_Rate,Weak_foot,...,Jumping,Heading,Shot_Power,Finishing,Long_Shots,Curve,Freekick_Accuracy,Penalties,Volleys,label
0,Cristiano Ronaldo,Portugal,LS,LW,185,80,Right,32,High / Low,4,...,95,85,92,93,90,81,76,85,88,1.0
1,Lionel Messi,Argentina,RW,RW,170,72,Left,29,Medium / Medium,4,...,68,71,85,95,88,89,90,74,85,1.0
2,Neymar,Brazil,LW,LW,174,68,Right,25,High / Medium,5,...,61,62,78,89,77,79,84,81,83,1.0
3,Luis Suárez,Uruguay,LS,ST,182,85,Right,30,High / Medium,4,...,69,77,87,94,86,86,84,85,88,0.0
4,Manuel Neuer,Germany,GK,GK,193,92,Right,31,Medium / Medium,4,...,78,25,25,13,16,14,11,47,11,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11600,Owain Fôn Williams,Wales,Sub,GK,193,77,Right,30,Medium / Medium,3,...,61,16,13,11,13,11,12,16,11,5.0
11783,Ricardo Reyes,Mexico,LB,LWB,181,66,Left,25,High / High,3,...,62,62,64,34,49,41,46,48,44,3.0
12330,Andraž Struna,Slovenia,RB,RB,185,72,Right,27,Medium / Medium,4,...,74,54,48,32,43,44,47,57,43,4.0
12655,Riley McGree,Australia,Sub,CDM,178,72,Left,18,Medium / Medium,3,...,51,44,50,50,52,49,43,44,40,3.0


In [ ]:
df_position["label"].value_counts()

4.0    88
5.0    80
2.0    70
1.0    65
0.0    42
3.0    21
Name: label, dtype: int64

La clase con menos personas es central defensa y la con más es defensa. 

In [ ]:
ct = ColumnTransformer(
    [
        (
            "Scaler",
            StandardScaler(),
            [
                "Height",
                "Weight",
                "Age",
                "Ball_Control",
                "Dribbling",
                "Marking",
                "Sliding_Tackle",
                "Standing_Tackle",
                "Aggression",
                "Reactions",
                "Interceptions",
                "Vision",
                "Composure",
                "Crossing",
                "Short_Pass",
                "Long_Pass",
                "Acceleration",
                "Speed",
                "Stamina",
                "Strength",
                "Balance",
                "Agility",
                "Jumping",
                "Heading",
                "Shot_Power",
                "Finishing",
                "Long_Shots",
                "Curve",
                "Freekick_Accuracy",
                "Penalties",
                "Volleys"
            ],
        ),
        (
            "OneHot",
            OneHotEncoder(sparse=False, handle_unknown="ignore"),
            ["Name", "Nationality", "National_Position", "Preffered_Foot", "Work_Rate"],
        ),
    ]
)

knn_pipe2 = Pipeline(
    [
        ("preprocesamiento", ct), 
        ("knn", KNeighborsClassifier(n_neighbors=3))
    ]
)

svc_pipe2 = Pipeline([
    ("preprocesamiento", ct), 
    ("svc", SVC(kernel="linear"))
])

svc_pipe_rbf2 = Pipeline([
    ("preprocesamiento", ct), 
    ("svc", SVC(kernel="rbf")
    )
])


In [ ]:
features = df_position.drop(columns=["Club_Position", "label"])
labels = df_position.loc[:, "label"]

display(features)
display(labels)

X_train, X_test, y_train, y_test = train_test_split(
    features, labels, test_size=0.33, shuffle=True, stratify=labels
)

,Name,Nationality,National_Position,Height,Weight,Preffered_Foot,Age,Work_Rate,Weak_foot,Skill_Moves,...,Agility,Jumping,Heading,Shot_Power,Finishing,Long_Shots,Curve,Freekick_Accuracy,Penalties,Volleys
0,Cristiano Ronaldo,Portugal,LS,185,80,Right,32,High / Low,4,5,...,90,95,85,92,93,90,81,76,85,88
1,Lionel Messi,Argentina,RW,170,72,Left,29,Medium / Medium,4,4,...,90,68,71,85,95,88,89,90,74,85
2,Neymar,Brazil,LW,174,68,Right,25,High / Medium,5,5,...,96,61,62,78,89,77,79,84,81,83
3,Luis Suárez,Uruguay,LS,182,85,Right,30,High / Medium,4,4,...,86,69,77,87,94,86,86,84,85,88
4,Manuel Neuer,Germany,GK,193,92,Right,31,Medium / Medium,4,1,...,52,78,25,25,13,16,14,11,47,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11600,Owain Fôn Williams,Wales,Sub,193,77,Right,30,Medium / Medium,3,1,...,61,61,16,13,11,13,11,12,16,11
11783,Ricardo Reyes,Mexico,LB,181,66,Left,25,High / High,3,2,...,65,62,62,64,34,49,41,46,48,44
12330,Andraž Struna,Slovenia,RB,185,72,Right,27,Medium / Medium,4,2,...,60,74,54,48,32,43,44,47,57,43
12655,Riley McGree,Australia,Sub,178,72,Left,18,Medium / Medium,3,2,...,70,51,44,50,50,52,49,43,44,40


0        1.0
1        1.0
2        1.0
3        0.0
4        5.0
        ... 
11600    5.0
11783    3.0
12330    4.0
12655    3.0
12737    0.0
Name: label, Length: 366, dtype: float64

In [ ]:
#ENTRENAMIENTO Y CLASIFICACION KNN
knn_pipe2.fit(X_train, y_train)
y_pred = knn_pipe2.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_pred, y_test))

[[12  1  1  0  0  0]
 [ 3 10  8  0  1  0]
 [ 2  8 10  0  3  0]
 [ 0  3  0  1  3  0]
 [ 0  4  1  0 24  0]
 [ 0  0  0  0  0 26]]
              precision    recall  f1-score   support

         0.0       0.86      0.71      0.77        17
         1.0       0.45      0.38      0.42        26
         2.0       0.43      0.50      0.47        20
         3.0       0.14      1.00      0.25         1
         4.0       0.83      0.77      0.80        31
         5.0       1.00      1.00      1.00        26

    accuracy                           0.69       121
   macro avg       0.62      0.73      0.62       121
weighted avg       0.72      0.69      0.70       121



In [ ]:
# Entrenamos
svc_pipe2.fit(X_train, y_train)

# Predecimos
y_pred = svc_pipe2.predict(X_test)

# Evaluamos
print("Matriz de confusión\n", confusion_matrix(y_pred, y_test))
print(classification_report(y_pred, y_test))

Matriz de confusión
 [[13  4  0  0  0  0]
 [ 1  7 12  0  2  0]
 [ 0 11  8  1  1  0]
 [ 0  0  1  5  2  0]
 [ 0  0  2  1 24  0]
 [ 0  0  0  0  0 26]]
              precision    recall  f1-score   support

         0.0       0.93      0.76      0.84        17
         1.0       0.32      0.32      0.32        22
         2.0       0.35      0.38      0.36        21
         3.0       0.71      0.62      0.67         8
         4.0       0.83      0.89      0.86        27
         5.0       1.00      1.00      1.00        26

    accuracy                           0.69       121
   macro avg       0.69      0.66      0.67       121
weighted avg       0.70      0.69      0.69       121



In [ ]:
# Entrenamos
svc_pipe_rbf2.fit(X_train, y_train)

# Predecimos
y_pred = svc_pipe_rbf2.predict(X_test)

# Evaluamos
print("Matriz de confusión\n", confusion_matrix(y_pred, y_test))
print(classification_report(y_pred, y_test))

Matriz de confusión
 [[12  1  2  0  0  0]
 [ 1  8  6  0  2  0]
 [ 1 12 11  2  1  0]
 [ 0  0  0  0  0  0]
 [ 0  1  4  5 26  0]
 [ 0  0  0  0  0 26]]
              precision    recall  f1-score   support

         0.0       0.86      0.80      0.83        15
         1.0       0.36      0.47      0.41        17
         2.0       0.48      0.41      0.44        27
         3.0       0.00      0.00      0.00         0
         4.0       0.90      0.72      0.80        36
         5.0       1.00      1.00      1.00        26

    accuracy                           0.69       121
   macro avg       0.60      0.57      0.58       121
weighted avg       0.75      0.69      0.71       121



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


De todos los clasificadores, los mejores son el SVC linela y el KNN. en el caso del KNN, como se puede notar en la matriz su matriz de confusión, la mayoría de los valores se encuentran en la diagonal de la matriz, lo que quiere decir que varios de los datos son clasificados correctamente. Sin emabrgo al ver los valores de la precision y recall, podemos notar que no puede clasificar correctamente a los datos etiquetados como 3, correspondientes a los defensa central. En cuando al SVC lineal, este logra clasificar un poco mejor a los datos del grupo 3, pero baja la precisión para otros grupos, aún así, la precisión en promedio de este clasificador es mejor al KNN. 


La clasificación de posicion fue mucho mejor lograda que la clasificación de los seleccionados nacionales, sin ambargo tampoco se puede decir que fue excelente.


## 1.3 Predicción de Sueldos [2 puntos]

<p align="center">
  <img src="https://media.giphy.com/media/3oEjHWtHAFcOKS4iA0/giphy.gif" width="460">
</p>

Queriendo ahondar aún más en el mercado del balompíe, Renacin, logra obtener (de una manera no muy formal) los sueldos de múltiples futbolistas y los guarda en el archivo ```sueldos.csv```. Con ellos les solicita que generen un regresor que les permita predecir el sueldo de los futbolistas en base a las características de los pichichis, esto, debido a su motivación por invertir y/o realizar especulación sobre los sueldos de jugadores.

Renacin es claro señalando que deben seguir utilizando la [guía](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html) y comenten cada uno de los pasos realizados, para obtener su regresión lineal. Señalándoles que no aceptara un $R^2$ inferior a 0.35 para el modelo solicitado.

Para esta parte usted tiene total libertad en la generación del regresor, la unica exigencia es que utilice un pipeline para generar la regresión y utilice la metrica $R^2$ para medir el rendimiento de esta.

**To-Do**:

- [ ] Explique en que consiste la métrica $R^2$
- [ ] Generar un pipeline para la regresión.
- [ ] Obtener un regresor con un $R^2$ superior a  $0.35$.
- [ ] Comente sus resultados y si es posible mejorar los resultados obtenidos. ¿Se necesitarían más datos o otros tipos de características o una combinación de ambos?

**Respuesta**

In [ ]:
### Código aquí ###

**Respuesta**:

```
Comente de resultados observados y si es posible mejorar los resultados.
```

# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana. Cualquier duda del laboratorio, no duden en contactarnos por mail o U-cursos.

<p align="center">
  <img src="https://media2.giphy.com/media/l1J3CbFgn5o7DGRuE/giphy.gif">
</p>

<br>
<center>
<img src="https://i.kym-cdn.com/photos/images/original/001/194/195/b18.png" width=100 height=50 />
</center>
<br>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>